In [2]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [3]:
dict_table= pickle.load(open('table','rb'))
df_historical_data = pd.read_csv('Clean Fifa all historical data.csv')
df_fixture_data = pd.read_csv('Clean fixture_data.csv')

df_historical_data

,Unnamed: 0,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,0,France,Mexico,1930,4,1,5
1,17,Uruguay,Argentina,1930,4,2,6
2,16,Uruguay,Yugoslavia,1930,6,1,7
3,15,Argentina,United States,1930,6,1,7
4,14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...,...
1039,563,Brazil,Costa Rica,2018,2,0,2
1040,564,Serbia,Switzerland,2018,1,2,3
1041,565,Serbia,Brazil,2018,0,2,2
1042,567,Germany,Mexico,2018,0,1,1


In [4]:
#slip into home / awawy
df_home= df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away= df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [5]:
#Rename 
df_home = df_home.rename(columns=({'HomeTeam': 'Team' ,'HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'}))
df_away= df_away.rename(columns=({'AwayTeam': 'Team' ,'HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'}))


In [6]:
#Team Streangth

df_team_strength = pd.concat([df_home,df_away], ignore_index= True ).groupby('Team').mean()
df_team_strength.sort_values('GoalsScored')


,GoalsScored,GoalsConceded
Team,,
Zaire,0.000000,4.666667
Dutch East Indies,0.000000,6.000000
Canada,0.000000,1.666667
Trinidad and Tobago,0.000000,1.333333
China,0.000000,3.000000
...,...,...
Turkey,1.923077,1.538462
West Germany,2.112903,1.241935
Germany,2.125000,0.928571


In [10]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
        
    else:
        return (0, 0)
      

In [11]:
predict_points('Bolivia','Brazil')

(0.002292074923096908, 2.6745300782662254)